In [1]:
import numpy as np
import pandas as pd

In [6]:
filepath = '/home/mohammed/GreyAtom/Python/Projects/datalytics/DataFiles/Hotels/FiltredDataFile.csv'
hotel = pd.read_csv(filepath)
hotel.head(2)

,Unnamed: 0,Hotel_Name,City_Name,000,001,002,003,004,005,006,...,247,248,249,250,251,252,253,254,256,Total
0,0,Tanner's,atlanta,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,13
1,1,Frijoleros,atlanta,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,14


In [7]:
#features = ['005','024','025']
features = ['076']
ones = list(np.ones(len(features),dtype=int))
ones

[1]

In [11]:
def find_hotel_with_features(hotel,features):
    bool_filter = []
    for values in hotel[features].values:
        bool_filter.append(list(values) == ones)
    return hotel[bool_filter].sort_values('Total',ascending=False).head(10)

In [12]:
find_hotel_with_features(hotel,features)

,Unnamed: 0,Hotel_Name,City_Name,000,001,002,003,004,005,006,...,247,248,249,250,251,252,253,254,256,Total
288,288,RITZ DINING ROOM,boston,0,0,0,0,0,0,0,...,0,0,0,2,0,0,1,0,0,28
4046,4046,SEASONS,washington_dc,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,28
3783,3783,KINKEAD'S,washington_dc,0,0,0,0,0,1,0,...,0,0,0,1,0,1,1,0,0,27
3853,3853,Jefferson Restaurant,washington_dc,0,0,0,0,0,1,0,...,0,1,0,1,0,1,1,0,0,26
704,704,PARKER'S,boston,0,0,0,0,0,0,0,...,0,0,0,2,0,0,1,0,0,26
3868,3868,THE COLONNADE,washington_dc,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,25
3985,3985,LAFAYETTE,washington_dc,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,24
3928,3928,Melrose,washington_dc,0,0,0,0,0,1,0,...,0,0,0,1,0,1,1,0,0,24
3838,3838,701,washington_dc,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,24
529,529,RITZ CAFE,boston,0,0,0,0,0,1,0,...,0,0,0,2,0,0,1,0,0,23
